# ntakt Basic Usage

ntakt is currently not available through remote maven repositories. To install into your local maven repository, run
```
./gradlew build publishToMavenLocal
```
in the project's root directory.

In [1]:
// set up dependencies
// use local maven repository; not yet deployed to remote maven repositories.
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")

// uncomment to search in your local maven repo
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("org.ntakt:ntakt:0.1.5-SNAPSHOT")

// uncomment to search in jitpack
// @file:DependsOn("org.ntakt:ntakt:main-SNAPSHOT")

%use lets-plot

Simply import all names from the `org.ntakt` package to get started:

In [2]:
import org.ntakt.*

Other imports

In [3]:
import kotlin.math.sqrt
import kotlin.random.Random

## Generate Data

Create ImgLib2 `ArrayImg`s with the `ntakt` object. The images are backed by Java primitive type arrays. Images can be created
 1. with dimensions only (populate with default primitive type array value, usually `0`)
 2. with dimensions and initializer that consumes linear index and type at voxel
 3. with dimensions and initializer that produces a primitive type value for linear index

Note: Use `flatStringRepresentation` only for small images.

In [4]:
// dims only (1)
println(ntakt.booleans(1, 2, 3).flatStringRepresentation)
// dims with initializer (2)
println(ntakt.argbs(1, 2, 3) { (it + 5) * 100 }.flatStringRepresentation)
// dims with initializer (3)
println(ntakt.unsignedLongs(1, 2, 3) { Long.MIN_VALUE }.flatStringRepresentation)
println(ntakt.floats(1, 2, 3) { i -> i.toFloat() + 0.5f }.flatStringRepresentation)
println(ntakt.doubles(1, 2, 3) { i -> i.toDouble() + 1.0 }.flatStringRepresentation)

ArrayImg [1x2x3]: [false, false, false, false, false, false]
ArrayImg [1x2x3]: [(r=0,g=1,b=244,a=0), (r=0,g=2,b=88,a=0), (r=0,g=2,b=188,a=0), (r=0,g=3,b=32,a=0), (r=0,g=3,b=132,a=0), (r=0,g=3,b=232,a=0)]
ArrayImg [1x2x3]: [9223372036854775808, 9223372036854775808, 9223372036854775808, 9223372036854775808, 9223372036854775808, 9223372036854775808]
ArrayImg [1x2x3]: [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
ArrayImg [1x2x3]: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]


Virtual images can be created without any data allocation. The values are generated on request. If no interval is specified, the domain of the image will be unbounded

In [5]:
val dims = longArrayOf(1L, 2L, 3L)
println(ntakt.zeros(3))
println(ntakt.zeros(dims.interval).flatStringRepresentation)

net.imglib2.util.ConstantUtils$1@56f408b4
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


You can restrict the bounds of any image (bound or unbound) to any interval (of the same dimensionality) with the bracket operator `[]`. Note that, for bounded images, the new bounds are not checked for consistency with existing bounds.

In [6]:
println(ntakt.ones(3))
println(ntakt.ones(3)[dims.interval].flatStringRepresentation)
println(ntakt.ones(dims.interval).flatStringRepresentation)
println(ntakt.ones(longArrayOf(1, 1, 1).interval).flatStringRepresentation)

net.imglib2.util.ConstantUtils$1@3a98e097
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
IntervalView [(0, 0, 0) -- (0, 0, 0) = 1x1x1]: [1.0]


For the inverse operation, bounded images can be extended with various strategies. In this example, the extended images are bounded to larger bounds than the original image to demonstrate the extension. One-dimensional images are used because the visualization with flat Strings will be more meaningful.

In [7]:
val dims = longArrayOf(5)
val extended = longArrayOf(-3, 7).intervalMinMax
val img = ntakt.doubles(*dims) { i -> 3.5 - abs(2.0 - i) }
println("Original image:             ${img.flatStringRepresentation}")
println("Extend with zeros:          ${img.extendZero()[extended].flatStringRepresentation}")
println("Extend with value:          ${img.extendValue(3.14)[extended].flatStringRepresentation}")
println("Extend with border value:   ${img.extendBorder()[extended].flatStringRepresentation}")
println("Extend mirror single:       ${img.extendMirrorSingle()[extended].flatStringRepresentation}")
println("Extend mirror double:       ${img.extendMirrorDouble()[extended].flatStringRepresentation}")

Original image:             ArrayImg [5]: [1.5, 2.5, 3.5, 2.5, 1.5]
Extend with zeros:          IntervalView [(-3) -- (7) = 11]: [0.0, 0.0, 0.0, 1.5, 2.5, 3.5, 2.5, 1.5, 0.0, 0.0, 0.0]
Extend with value:          IntervalView [(-3) -- (7) = 11]: [3.14, 3.14, 3.14, 1.5, 2.5, 3.5, 2.5, 1.5, 3.14, 3.14, 3.14]
Extend with border value:   IntervalView [(-3) -- (7) = 11]: [1.5, 1.5, 1.5, 1.5, 2.5, 3.5, 2.5, 1.5, 1.5, 1.5, 1.5]
Extend mirror single:       IntervalView [(-3) -- (7) = 11]: [2.5, 3.5, 2.5, 1.5, 2.5, 3.5, 2.5, 1.5, 2.5, 3.5, 2.5]
Extend mirror double:       IntervalView [(-3) -- (7) = 11]: [3.5, 2.5, 1.5, 1.5, 2.5, 3.5, 2.5, 1.5, 1.5, 2.5, 3.5]


## Materializing RandomAccessibleIntervals

Many of the `RandomAccessibleIntervals` are views that are evaluated on request for each voxel. The data is not materialized and generated with each access. For expensive operations that will be accessed more than once, it may be beneficial to materialize or cache the results. `ntakt` provides to options:
 1. `materialize` the `RandomAccessibleInterval` into an `ArrayImg` that is backed by a primitive type array
 2. `cache` the `RandomAcecssibleInterval` into a `CachedCellImg` with various caching strategies.

In [8]:
val dims = longArrayOf(10)
val img = ntakt.doubles(*dims) { i -> i.toDouble() }
val bellCurve = (-((img - 5.0) `**` 2.0) / 4.0).exp()
val sqrt = bellCurve.convert(bellCurve.type) { s, t -> println("square-rooting $s"); t.setReal(sqrt(s.realDouble)) }
sqrt.flatStringRepresentation
println()
sqrt.flatStringRepresentation
println()
sqrt.flatStringRepresentation
println()
println("Materializing")
val materialized = sqrt.materialize()
materialized.flatStringRepresentation
materialized.flatStringRepresentation
materialized.flatStringRepresentation
println()
println("Caching with block size 5")
val cached = sqrt.cache(5)
cached.flatStringRepresentation
println()
cached.flatStringRepresentation
cached.flatStringRepresentation


square-rooting 0.00193045413622771
square-rooting 0.018315638888734186
square-rooting 0.10539922456186435
square-rooting 0.36787944117144233
square-rooting 0.7788007830714049
square-rooting 1.0
square-rooting 0.7788007830714049
square-rooting 0.36787944117144233
square-rooting 0.10539922456186435
square-rooting 0.018315638888734186

square-rooting 0.00193045413622771
square-rooting 0.018315638888734186
square-rooting 0.10539922456186435
square-rooting 0.36787944117144233
square-rooting 0.7788007830714049
square-rooting 1.0
square-rooting 0.7788007830714049
square-rooting 0.36787944117144233
square-rooting 0.10539922456186435
square-rooting 0.018315638888734186

square-rooting 0.00193045413622771
square-rooting 0.018315638888734186
square-rooting 0.10539922456186435
square-rooting 0.36787944117144233
square-rooting 0.7788007830714049
square-rooting 1.0
square-rooting 0.7788007830714049
square-rooting 0.36787944117144233
square-rooting 0.10539922456186435
square-rooting 0.018315638888734

CachedCellImg [10]: [0.04393693362340743, 0.1353352832366127, 0.32465246735834974, 0.6065306597126334, 0.8824969025845955, 1.0, 0.8824969025845955, 0.6065306597126334, 0.32465246735834974, 0.1353352832366127]

## Arithmetic operations
`RandomAccessible`s and `RandomAccessibleInterval`s support arithmetic operations with other `RandomAccessible`s and `RandomAccessibleInterval`s and scalar values. Results of arithmetic operations are lazy views that are evaluated on request for each voxel. For expensive arithmetic operations, it may be beneficial to materialize these views into and `Img` if they are accessed multiple times.

In [9]:
val dims = longArrayOf(5)
val img = ntakt.doubles(*dims) { i -> i + 1.0 }
val extended = img.extendBorder()
println("img: ${img.flatStringRepresentation}")
println("img squared: ${(img `**` 2.0).flatStringRepresentation}")
println("img + img: ${(img + img).flatStringRepresentation}")
println("gradient: ${(extended.translate(+1L) - extended.translate(-1L))[img].flatStringRepresentation}")
println("bell curve: ${(-((img - 3.1) `**` 2.0) / 6.25).exp().flatStringRepresentation}") 

img: ArrayImg [5]: [1.0, 2.0, 3.0, 4.0, 5.0]
img squared: net.imglib2.converter.read.ConvertedRandomAccessibleInterval@5b646928: [1.0, 4.0, 9.0, 16.0, 25.0]
img + img: net.imglib2.converter.read.BiConvertedRandomAccessibleInterval@3e0730cd: [2.0, 4.0, 6.0, 8.0, 10.0]
gradient: IntervalView [(0) -- (4) = 5]: [-1.0, -2.0, -2.0, -2.0, -1.0]
bell curve: net.imglib2.converter.read.ConvertedRandomAccessibleInterval@1eac6d9a: [0.49381219803334614, 0.8239874333317032, 0.9984012793176064, 0.8784467393499313, 0.5612437364432349]


In [10]:
val dims = longArrayOf(1000)
val img = ntakt.doubles(*dims) { it.toDouble() }
println((img - 500.0).type::class)
val bellCurve = (-((img - 500.0) `**` 2.0) / 40000.0).exp()
val d = mapOf<String, Any>(
    "X" to img.map { it.realDouble },
    "bell curve" to bellCurve.flatIterable.map { it.realDouble }
)
val p = lets_plot(d)
p + geom_line { x = "X"; y = "bell curve" } + ggsize(800, 500)

class net.imglib2.type.numeric.real.DoubleType
